In [4]:
import torch
print(torch.cuda.is_available())

True


In [5]:
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertForMultipleChoice, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import os

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:184: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 950M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 5.2.
    
  warnings.warn(


In [6]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [7]:
path = "../../data/processed"
dataset = load_dataset(path)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 97139
    })
    test: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 5113
    })
})

In [ ]:
print(dataset["train"][0])

{'text': 'Prázdná hala a komorní atmosféra na Spartě. Proč hokejová LM netáhne? Zajímavý zápas na ledě, ale velká bída v hledišti. Tak vypadalo první utkání play off hokejové Ligy mistrů mezi domácí Spartou a Kärpätem Oulu, které nakonec skončilo remízou 1:1. Tu ovšem sledovalo pouze 2 288 diváků, atmosféra tak byla komorní. „Je škoda, že lidi nechodí, protože je to super soutěž,“ mrzelo útočníka Sparty Lukáše Pecha.', 'encoded_text': 13}


In [ ]:
MODEL_NAME = "distilbert-base-uncased"

In [ ]:
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_quant_type="fp4",  # Use FP4 quantization
    bnb_4bit_compute_dtype=torch.float16,  # Set computation data type to float16 for reduced memory usage and potentially faster computation
    llm_int8_enable_fp32_cpu_offload=False,  # Disable CPU offload for INT8 operations
    llm_int8_has_fp16_weight=True  # Use FP16 weights for better fine-tuning stability
)

In [ ]:
df = dataset['train'].to_pandas()
count_of_categories = df['encoded_text'].nunique()
print(count_of_categories)

24


In [ ]:
model = DistilBertForMultipleChoice.from_pretrained(
    MODEL_NAME,
    quantization_config=bitsandbytes_config,
    attn_implementation="flash_attention_2",
    num_labels=count_of_categories
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`